In [4]:
import numpy as np
import torch
from tqdm import tqdm
from recbole.quick_start.quick_start import load_data_and_model
from recbole.utils.case_study import full_sort_topk, full_sort_scores
import pandas as pd
import os
model_path='EASE-Feb-15-2024_04-53-56.pth'
topk=10
save_path='submission.csv'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=os.path.join('saved',model_path))

df = pd.read_csv('/home/minseo/Project/Recbole/data/train/train_ratings.csv')
user_grp = dict(df.groupby('user').item.apply(list))

submission = pd.read_csv('/home/minseo/Project/Recbole/data/eval/sample_submission.csv')
sub_user_idx = submission['user'].unique()
sub_user_idx = np.array(sub_user_idx,dtype=str)
uid_series = dataset.token2id(dataset.uid_field, sub_user_idx)
total_topk_score, total_topk_iid_list = torch.zeros_like(torch.Tensor(31360, topk)), torch.zeros_like(torch.Tensor(31360, topk))

for idx in tqdm(range(0,len(uid_series))):
    need_inf = dataset.token2id(dataset.iid_field, np.array(user_grp[int(sub_user_idx[idx])],dtype=str))
    mask = [True if i in need_inf else False for i in range(0,6808)]
    scores = full_sort_scores(np.array([uid_series[idx]]),model,valid_data,config['device'])
    new_scores=scores.cpu().masked_fill(torch.from_numpy(np.array(mask)),float('-inf'))    
    total_topk_score[idx] = torch.topk(new_scores,topk)[0]
    total_topk_iid_list[idx] = torch.topk(new_scores,topk)[1]
    
int_iid = total_topk_iid_list.to(torch.int64)
external_item_list = dataset.id2token(dataset.iid_field, int_iid.cpu())
external_item_list = external_item_list.flatten()
df = pd.DataFrame({'user': np.repeat(sub_user_idx, topk), 'item': external_item_list})
df.to_csv(save_path,index=False)


15 Feb 21:40    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ML
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.999, 0.001, 0]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
fiel

KeyboardInterrupt: 

In [ ]:
saved_val = valid_data

tensor([[   -inf, -0.0451,  0.0102,  ...,  0.0087, -0.0105, -0.0103]])

In [15]:
for idx,i in enumerate(valid_data):
    if idx == 1 :
        print(i[1][0].shape, i[1][1].shape)
    if idx == 0 :
        print(i[1][0].shape, i[1][1].shape)
    else:
        break

torch.Size([375]) torch.Size([375])
torch.Size([179]) torch.Size([179])


In [ ]:
model_path='SASRec-Feb-15-2024_21-03-50.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=os.path.join('saved',model_path))

In [ ]:
df = pd.read_csv('/home/minseo/Project/Recbole/data/train/train_ratings.csv')
user_grp = dict(df.groupby('user').item.apply(list))

submission = pd.read_csv('/home/minseo/Project/Recbole/data/eval/sample_submission.csv')
sub_user_idx = submission['user'].unique()
sub_user_idx = np.array(sub_user_idx,dtype=str)
uid_series = dataset.token2id(dataset.uid_field, sub_user_idx)
total_topk_score, total_topk_iid_list = torch.zeros_like(torch.Tensor(31360, topk)), torch.zeros_like(torch.Tensor(31360, topk))

for idx in tqdm(range(0,len(uid_series))):
    need_inf = dataset.token2id(dataset.iid_field, np.array(user_grp[int(sub_user_idx[idx])],dtype=str))
    mask = [True if i in need_inf else False for i in range(0,6808)]
    scores = full_sort_scores(np.array([uid_series[idx]]),model,saved_val,config['device'])
    new_scores=scores.cpu().masked_fill(torch.from_numpy(np.array(mask)),float('-inf'))    
    total_topk_score[idx] = torch.topk(new_scores,topk)[0]
    total_topk_iid_list[idx] = torch.topk(new_scores,topk)[1]
    
int_iid = total_topk_iid_list.to(torch.int64)
external_item_list = dataset.id2token(dataset.iid_field, int_iid.cpu())
external_item_list = external_item_list.flatten()
df = pd.DataFrame({'user': np.repeat(sub_user_idx, topk), 'item': external_item_list})
df.to_csv(save_path,index=False)